In [3]:
path='../input/dataset/'

In [4]:
import pandas as pd
import numpy as np

In [5]:
train=pd.read_csv(path+"train.csv")
test=pd.read_csv(path+"test.csv")

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [7]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
lst = train['comment_text'].tolist()
lst[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [9]:
import re
from bs4 import BeautifulSoup
def clean(comment):
  comment = comment.lower() #lowercasing
  comment = re.sub(r"http\S+", "", comment) #removing URLs
  soup = BeautifulSoup(comment, 'lxml') #removing tags
  comment = soup.get_text()
  comment = comment.strip()
  comment = comment.split()
  comment = ' '.join(comment)
  return comment

In [10]:
lst1 = []
for i in range(len(lst)):
    lst1.append(clean(lst[i]))

In [11]:
lst1[0]

"explanation why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.89.205.38.27"

In [12]:
#Expandig English language contractions
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [13]:
for i in range(len(lst1)):
    lst1[i] = decontracted(lst1[i])
    lst1[i] = re.sub("\S*\d\S*", "", lst1[i]).strip() #Remove words with numbers
    lst1[i] = re.sub('[^A-Za-z0-9]+', ' ', lst1[i]) #Remove special characters

In [14]:
lst1[0]

'explanation why the edits made under my username hardcore metallica fan were reverted they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired'

In [15]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [16]:
comments = []
for i in range(len(lst1)):
    lst1[i] = ' '.join(e.lower() for e in lst1[i].split() if e.lower() not in stopwords)
    comments.append(lst1[i].strip())

In [17]:
len(comments)

159571

In [18]:
list_of_sentance = []
for ele in comments:
  list_of_sentance.append(ele.split())

In [19]:
want_to_train_w2v = True

if want_to_train_w2v:
    # min_count = 5 considers only words that occured atleast 2 times
    w2v_model=Word2Vec(list_of_sentance,min_count=2,vector_size=100, workers=8)
    print(w2v_model.wv.most_similar('great'))
    print('='*50)
    print(w2v_model.wv.most_similar('worst'))

[('excellent', 0.6757434010505676), ('wonderful', 0.6285920143127441), ('fantastic', 0.5850906372070312), ('good', 0.5593225359916687), ('brilliant', 0.5412752628326416), ('helped', 0.5381186008453369), ('nice', 0.5185155272483826), ('amazing', 0.49778029322624207), ('leden', 0.4888884127140045), ('terrible', 0.4795156419277191)]
[('dangerous', 0.6092609167098999), ('dumbest', 0.5943079590797424), ('terrible', 0.5911266207695007), ('sees', 0.5708798766136169), ('becoming', 0.5705922842025757), ('treating', 0.5683978199958801), ('ever', 0.5679754614830017), ('foolish', 0.5638439655303955), ('laughable', 0.5525754690170288), ('sad', 0.551774799823761)]


In [20]:
w2v_words = list(w2v_model.wv.key_to_index)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  81721
sample words  ['not', 'article', 'page', 'wikipedia', 'talk', 'would', 'please', 'no', 'one', 'like', 'see', 'also', 'think', 'know', 'people', 'edit', 'use', 'articles', 'time', 'may', 'thanks', 'even', 'get', 'user', 'make', 'could', 'good', 'well', 'information', 'want', 'deletion', 'sources', 'way', 'name', 'first', 'help', 'pages', 'new', 'wp', 'image', 'editing', 'source', 'go', 'need', 'section', 'say', 'fuck', 'edits', 'thank', 'made']


In [21]:
model = TfidfVectorizer()
tf_idf_matrix = model.fit_transform(comments)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [22]:
from tqdm import tqdm
# TF-IDF weighted Word2Vec
tfidf_feat = model.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence is stored in this list
row=0;
for sent in tqdm(list_of_sentance): # for each sentence 
    sent_vec = np.zeros(100) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence
    for word in sent: # for each word in a sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
            #  tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

100%|██████████| 159571/159571 [3:05:08<00:00, 14.36it/s]  


In [23]:
len(tfidf_sent_vectors)

159571

In [29]:
tfidf_sent_vectors[2]

array([ 0.41935966, -0.12742983, -0.48621123,  0.24537596,  0.09177995,
       -0.53996509,  0.99948687,  0.61016311,  0.33305385, -1.02187728,
        0.6514496 ,  0.43143233, -0.0912295 , -0.38507373,  0.08718207,
       -0.00412808, -0.1234714 , -0.12427172, -0.32407497, -0.13066183,
        0.32003298,  0.42522248,  0.48702434,  0.17302681,  0.66575449,
       -0.60830376, -0.85628014,  0.95604146,  0.52375813,  0.55560464,
       -0.083864  ,  0.38751514,  0.59229068,  0.49635678, -0.2933983 ,
        0.56638688,  0.09637144, -0.08100939, -0.77183406,  0.16834248,
        0.14154881, -0.10794321, -0.30248643, -0.31483727, -0.39063364,
       -0.52479099, -0.72075963,  0.03107077,  0.58920639, -0.22358562,
        0.02153007, -0.2223117 ,  0.11152225,  0.44778301, -0.52953696,
       -0.08493979,  1.31505058,  0.42008189,  0.28371861,  1.11199182,
       -0.56863546,  0.23625619, -0.03916826, -0.81548811,  0.33615425,
        0.34466594,  0.55053244, -0.35347903, -0.91617614,  0.25

In [25]:
type(tfidf_sent_vectors)

list

In [26]:
train['vector'] = tfidf_sent_vectors

In [27]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,vector
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[0.005933340688496345, 0.07003178458612487, -0..."
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[-0.25081059821595164, 0.016738989372669426, 0..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[0.41935966092030025, -0.12742982526573313, -0..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[0.32448252127970073, -0.4088936745209482, -0...."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[-0.04981408767189175, 0.6822151862510721, -0...."


In [30]:
train.to_csv("preprocessed_data.csv")